## A wrapper for OPA within AQUA

In [1]:
from aqua import OPAgenerator

Initialize the OPA genrator class, feeding it with the most of the information you need. Which variable, which model source and where to store the data

In [3]:
opaopa = OPAgenerator(model="IFS", exp="tco2559-ng5", source="ICMGG_atm2d", var='2t', frequency='daily',
            outdir='/home/b/b382076/scratch/one_pass', tmpdir='/home/b/b382076/scratch/one_pass', definitive=False)


2023-05-09 17:36:51 :: opa_generator :: WARNING  -> IMPORTANT: no file will be created, this is a dry run
2023-05-09 17:36:51 :: opa_generator :: WARNING  -> Variable(s) to be processed: 2t
2023-05-09 17:36:51 :: create_folder :: WARNING  -> Folder /home/b/b382076/scratch/one_pass/IFS/tco2559-ng5/daily already exists
2023-05-09 17:36:51 :: create_folder :: WARNING  -> Folder /home/b/b382076/scratch/one_pass already exists


Then you can retrieve the data using the Reader. This is based on a streaming generator so far since we are using data from NextGEMS, will be later replaced with direct access from FDB. It also gave a look at the data so that it can infer the timedelta

In [4]:
opaopa.retrieve()

2023-05-09 17:37:08 :: opa_generator :: WARNING  -> Getting the timedelta to inform the OPA...
2023-05-09 17:37:18 :: opa_generator :: WARNING  -> Timedelta is 60


This will launch the OPA computation, if the defitinive flag is true it will take ages and probably break the notebook. Otherwise print only the sequential access to the data from the Reader

In [5]:
opaopa.compute()

2023-05-09 17:37:44 :: opa_generator :: WARNING  -> Setting up OPA at daily frequency for variable 2t...
2023-05-09 17:37:44 :: opa_generator :: WARNING  -> Initializing the streaming generator...
2023-05-09 17:37:50 :: opa_generator :: WARNING  -> start_date: 2020-01-20T00:00:00.000000000 stop_date: 2020-01-22T23:00:00.000000000
2023-05-09 17:37:50 :: opa_generator :: WARNING  -> start_date: 2020-01-23T00:00:00.000000000 stop_date: 2020-01-25T23:00:00.000000000
2023-05-09 17:37:50 :: opa_generator :: WARNING  -> start_date: 2020-01-26T00:00:00.000000000 stop_date: 2020-01-28T23:00:00.000000000
2023-05-09 17:37:50 :: opa_generator :: WARNING  -> start_date: 2020-01-29T00:00:00.000000000 stop_date: 2020-01-31T23:00:00.000000000
2023-05-09 17:37:51 :: opa_generator :: WARNING  -> start_date: 2020-02-01T00:00:00.000000000 stop_date: 2020-02-03T23:00:00.000000000
2023-05-09 17:37:52 :: opa_generator :: WARNING  -> start_date: 2020-02-04T00:00:00.000000000 stop_date: 2020-02-06T23:00:00.000

You can then add an entry to the catalog so that you can access these data from somewhere else

In [6]:
opaopa.create_catalog_entry()

2023-05-09 17:40:01 :: opa_generator :: WARNING  -> Creating catalog entry IFS tco2559-ng5 tmp-opa-daily


This remove the entries (and in the future also the files), once all the computation have been safely concluded

In [7]:
opaopa.safe_clean()

2023-05-09 17:40:03 :: opa_generator :: WARNING  -> Removing catalog entry IFS tco2559-ng5 tmp-opa-daily
